In [1]:
# imports
import numpy as np
import pandas as pd

df = pd.read_csv('MetObjects.csv')

C:\Users\OndraS\anaconda3\envs\s\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,10,11,12,13,14,15,22,34,35,36,37,38,39,40,41,42,43,44,46) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Homework 1 - data validation & cleaning (deadline 31. 10. 2020)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can!

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency (i.e. that same things are represented in the same way) of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features. By integrity we mean correct logical relations between features (e.g. female names for females only).
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your method).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 16 points.** However, all your steps and following code have to be commented! 

To earn **two points** for extra activity you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, she can (!) give you another chance to rework your homework and to obtain more points. However, this is not a given, so do your best! :)

##### Prvni pohled na  data

In [2]:
# display(df.shape) # row and cols cnt
# df.nunique()
# df.info() # cool, top info

## 2. Check consistency (i.e. that same things are represented in the same way) of at least three features where you expect problems (include "Object Name" feature).

Nejprve zacnu sloupcem hodnot 'Object Name'

In [3]:
print(df['Object Name'].nunique())
# display(df['Object Name'].head())
# for typ in df['Object Name'].unique():
#     print(typ)
len(df[df['Object Name'].isnull()])

28076


1354

Vypada to, ze sloupec 'Object Name' lze interpretovat jako jakysi typ objektu, protoze hodnoty jsou casto velmi genericke.
V databazi je sice ~ 28 tisic typu objektu, ale vzhledem k tomu, ze celkem je v datasetu skoro pul milionu zaznamu tak si myslim, ze upraveni typu tohoto sloupce na kategoricky je vice nez vhodne.

* Pri prochazeni nazvu jednotlivych typu jsem si vsiml, ze pomerne casto se vyskytuji typy ve stylu '12 zidli', '13 zidli' apod. Proto jsem se rozhodl nejprve modifikat hodnoty v sloupcich tak, ze pokud prvni posloupnost znaku tvori cislovku, tak ji odeberu z nazvu typu.
* Dale jsem si vsiml, ze obcas jsou nazvy prilis dlouhe, protoze zabihaji do detailu specifickych predmetu. Rozhodl jsem se proto vsechny nazvy typu, ktere obsahuji vice nez 30 znaku oriznout a to tak, ze z nich zachovam pouze prvni dve slova
* Pri vypisu jednotlivych hodnot jsem nabyl pocitu, ze se obcas vyskytuji prazdne hodnoty(jedna mezera, ci dve atd) - tak jsem je pro jistotu nahradil za NaN. Po vypisu zmenenych hodnot se ukazalo, ze to byla nejspis pouze chyba vypisu
* Vice nez 1300 radku ma v sloupci NaN, pro tyto sloupce jsem vytvoril novy typ do categorie a to 'Other'
* Dalsiho zredukovani typu jsem dosahl pri pretvoreni vsech hodnot do lowercasu
* Jako posledni krok jsem cely sloupec prejmenoval na 'ObjectType'

In [4]:
def replaceWhiteSpaceWithNan(val):
    if str(val).isspace() or len(str(val)) is 0:
        return np.nan
    return str(val)

In [5]:
df['Object Name'] = df['Object Name'].apply(replaceWhiteSpaceWithNan)
df['Object Name'].fillna('Other', inplace=True)

In [6]:
def removeIntFromFirstPosition(val):
    if val.split()[0].isnumeric():
        return val.split(' ', 1)[1]
    return val

In [7]:
df['Object Name'] = df['Object Name'].apply(removeIntFromFirstPosition)

In [8]:
def trimTooLongTypeNames(val):
    if len(val) > 30:
        return val.split()[0]
    return val

In [9]:
df['Object Name'] = df['Object Name'].apply(trimTooLongTypeNames)

In [10]:
def replaceWhiteSpaceWithNan(val):
    if val.isspace() or len(val) is 0:
        return np.nan
    return val

In [11]:
df['Object Name'] = df['Object Name'].apply(replaceWhiteSpaceWithNan)

In [12]:
def transformToLowerCase(val):
    return str(val).lower()

In [13]:
df['Object Name'] = df['Object Name'].apply(transformToLowerCase)
df['Object Name'].fillna('Other', inplace=True)

In [14]:
df.rename(columns={'Object Name': 'ObjectType'}, inplace=True)
# vytvorim novou categorii z dostupnych unique hodnot
objName_category = pd.api.types.CategoricalDtype(categories=df['ObjectType'].unique(), ordered=True)
df['ObjectType'] = df['ObjectType'].astype(objName_category)

Dalsi nekonzistence, kterou jsem objevil byla ve sloupci 'Object Number'. Jako identifikator jednotlivych sloupcu slouzi sloupec 'Object ID', ktery obsahuje unikatni ID pro kazdy objekt. Sloupec 'Object Number' podle me ma obsahovat interni ID. Aby byly jednotlive sloupci s IDcky prvku konzistentni, meli by byt vsechny typu int.
* Hodnoty ve sloupci jsou velmi ruznorode, ve vetsine pripadu se jednalo o cisla, ale casto do nich byli nelogicky pridany ruzne znaky. Proto jsem se rozhodl, ze ze sloupcu vyparsuji pouze ciselne hodnoty a ty pouziji jako vyslednou hodnotu.
* Pokud se vyskytovala hodnota s pouze slovnim oznacenim, povazoval jsem to (i vzhledem k nazvu sloupce) za chybu a nahradil je hodnotou -1. U nekterych sloupcu byli hodnoty ve tvaru '123.45.67a' a '123.45.67b'. Tyto hodnoty podle meho symbolizuji nejaky vzajemny vztah mezi predmety a rozhodl jsem se jim nechat ve vysledku stejnou hodnotu (pro zadany priklad by tedy oba radky skoncili s hodnotou '1234567').
* V 35 pripadech jsem narazil na to, ze hodnoty ve sloupci obsahovaly ohromne mnozstvi cisel. Vysledne cislo by se neveslo do int64, coz je nejvetsi velikost int typu, ktery pandas podporuje. Dane radky jsem oznacil a vymazal je z datasetu. Bylo jich jen velmi male mnozstvi, takze si myslim, ze to je dobra volba.

In [15]:
def parseInt(val):
    val = str(val)
    intStr = ''
    for c in val:
        if c.isnumeric():
            intStr += c
    if len(intStr) is 0:
        return -1
    if len(intStr) > 18:
        return -2
    return int(intStr)/2

In [16]:
df['Object Number'] = df['Object Number'].apply(parseInt)
df['Object Number'] = df['Object Number'].astype('int64')
df = df[df['Object Number'] != -2]

Dalsi nekonzistenci tvoril sloupec 'Gallery Number', ktery ma jiste predstavovat, v jake galerii se dany objekt nachazi.
* Tento objekt by mel mit prirozene ciselne id po vzoru jinych sloupcu. Ale nema. Je tomu tak kvuli 5 hodnotam mistnosti, ktere maji misto cisla mistnosti jeji nazev. Vzhledem k tomu, ze jich bylo pouze 5 unikatnich, tak jsem si nasel cisla mistnosti, ktera dosud nebyla zabrana a vytvoril sem dictionary, ktery tyto string nazvy mistnosti mapuje na nalezena nepouzita cisla mistnosti
* Nektera cisla mistnosti jsou take zadana jako cisla s desetinou pozici, ovsem za desetinou teckou je vzdy nula, takze je zbytecne je v teto podobe mit. Odebral jsem tedy z cisel desetinou cast a cely sloupec jsem uspesne premenil na ciselny datovy typ int.
* Pri predpokladu, ze cisla galerie maji pouze predmety, ktere je potreba trackovat, doplnim na pozici NaNu 0, jako hodnotu signalizujici, ze predmet zatim nebyl umisten.

_pozn: Tento sloupec bych zvazil i odebrat jelikoz ve vetsine datasetu obsahuje pouze NaN hodnotu_

In [17]:
roomNameToId = {}
def checkNonNumericValue(val):
    if type(val) is int:
        return val
    if val is np.nan or val is None:
        return 0
    if type(val) is float:
        if np.isnan(val):
            return 0
        return int(val)
    if val.isnumeric():
        return val
    if val in roomNameToId.keys():
        return roomNameToId[val]
    else:
        # find possible roomId
        while True:
            nmb = np.random.randint(1,1000)
            if len(df[df['Gallery Number'] == nmb]) is 0:
                roomNameToId[val] = nmb
                return nmb

In [18]:
# df['Gallery Number'].unique()
df['Gallery Number'] = df['Gallery Number'].apply(checkNonNumericValue)
df['Gallery Number'] = df['Gallery Number'].astype('int32')

## 3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features. By integrity we mean correct logical relations between features (e.g. female names for females only)

Pri zkoumani dat me napadl tento potencionalni problem: Hodnota v sloupci 'Object End Date' (u niz predpokladam (i na zaklade ostatnich sloupcu), ze ma reprezentovat rok dokonceni dila) by nemela byt vetsi nez hodnota v sloupci 'Artist End Date' (predpokladam, ze reprezentuje datum umrti umelce). Myslim, ze by nemela byt vetsi, protoze mi logicky nesedi, aby dilo bylo dokoncene po smrti autora, i kdyz verim, ze v nekterych pripadech by mi nekdo tvrdil, ze takova situace muze nastat. Nemohu zarucit, ze tato myslenka je spravna, jelikoz nikde nejsou hodnoty ve sloupcich presne identifikovany, ale pokousel jsem se vymyslet neco vlastniho
* 'Object End Date' ma ciselne hodnoty, 'Artist End Date' ma hodnoty pomerne rozbite - ve vetsine pripadu obsahuje rok, ale obcas obsahuje napr. '1993|84945' (proste rok + chaos). Pokusil jsem se co nejrozumneji opravit a ziskat spravne roky z tohoto sloupce
* Nasledne jsem zkontroloval, zda se mnou ocekavany problem integrity v datech vyskytuje
    * Ano vyskytuje a to celkem v 25 423 pripadech, coz je pomerne vysoke cislo, ale vzhledem k tomu, jaky neporadek v datech je, tak se teto velke hodnote ani moc nedivim
    
_pozn: NaN hodnoty jsem kvuli prevodu do datoveho typu int oznacil dost specifickou hodnotou, kterou zadny radek v datasetu nemel. Jsem si vedom, ze to neni nejlepsi reseni, ale nenapadlo me, jakou jinou zvolit_

In [19]:
from datetime import date
current_date = date.today() 

def parseYear(val):
    if val is np.nan:
        return -666666
    val = str(val)
    result = ''
    negative = False
    if val[0] == '-':
        negative = True
        result = '-'
        val = val[1:]
        
    while len(result) <=5 and len(val) > 0:
        if val[0].isnumeric():
            result += val[0]
            val = val[1:]
        elif len(result) > 0 and not negative or negative and len(result) > 1:
            break
        else:
            val = val[1:]
    if int(result) >= current_date.year:
        return -666666
    return int(result)

In [20]:
df['Artist End Date'] = df['Artist End Date'].apply(parseYear)
df['Artist End Date'] = df['Artist End Date'].astype('int64')

In [21]:
print(len(df[(df['Object End Date'] > df['Artist End Date'])][['Object End Date', 'Artist End Date']]))

281104


## 4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime

#### Behem bodu 2) jsem zmenil jiz 3 priznaky (2x na kategorickou a 1x na numerickou hodnotu)
#### Behem bodu 3) jsem zmenil jeden sloupec na numerickou hodnotu

Pri dalsim zkoumani jsem narazil na nekolik sloupcu, u kterych jsem provedl jednoduche upravy pro zlepseni datasetu
* Sloupec 'Department' ma pouze 19 ruznych hodnot -> categorizuji
* Ve sloupci 'Artist Gender' jsou velmi rozbité hodnoty, z nichž soudím, že pokud se v změti znaku vyskytuje slovo 'female', pak se jednalo o zenu, v opacnem pripade o muze (soudim tak podle toho, ze pri vypisu unikatnich hodnot v tomto sloupci se vyskytuji pouze slova 'female' a nebo jen zmeti znaku)
    * Data v tomto sloupci tedy opravim a nasledne kategorizuji
* Hodnotu ve sloupci 'Repository' maji vsechny radky vyplnenou, ale stejnou! Odebiram sloupec
* Sloupec 'Metadata Date' je uplne nevyplneny - odebiram
* Sloupec 'Classification' jde categorizovat

In [22]:
def decideGender(val):
    val = str(val)
    if 'female' in val.lower():
        return 1
    return 0

In [23]:
# department column
department_category = pd.api.types.CategoricalDtype(categories=df['Department'].unique(), ordered=True)
df['Department'] = df['Department'].astype(department_category)
# Artist Gender column
gender_category = pd.api.types.CategoricalDtype(categories=[0, 1], ordered=True)
df['Artist Gender'] = df['Artist Gender'].apply(decideGender).astype(department_category)
# remove Repository and Metadata Date columns
df.drop(columns=['Repository', 'Metadata Date'], inplace=True)
# Classification column
df['Classification'].fillna('Other', inplace=True)
classif_category = pd.api.types.CategoricalDtype(categories=df['Classification'].unique(), ordered=True)
df['Classification'] = df['Classification'].astype(classif_category)

* Jeste zbyva prevest jeden sloupec do datetime formatu
    *  Rozhodl jsem se prevest hodnoty ve sloupci 'Artist End Date'
    * _pozn.: bohuzel v tomto sloupci je spousta hodnot NaN. Hodnoty pred rokem 1677 totiz neumi pandas prevest na datum, proto jsem je nechal nahradit hodnotou NaT. Nemyslim si, ze se jedna o nejlepsi zpusob a konverzi, ale docela dlouhy cast sem stravil ruznymi pokusy o vyparsovani a prevedeni na datum, ale neuspesne. V jinych sloupcich byla data obdobne 'oskliva'_

In [24]:
df['Artist End Date'] = pd.to_datetime(df['Artist End Date'], format='%Y', errors='coerce')

## 5. Find some outliers (describe your method)
* Ve sloupci 'Medium' se nachazi ruzne hodnoty a popisy, ovsem 4 hodnoty presahuji delku 1000 znaku, coz si nemyslim, ze je spravne. Urcite se musi jednat o chyby, ktere by mely byt opraveny. Po vypsani textu jsem se v tom ujistil:
    * jeden je z 90 % tvoren kopiemi posloupnosti znaku '\r\n'
    * druhy obsahuje celkem 66 kopii (jsou i ocislovane) vety 'Pen and brown ink, brush and brown and blue wash.\r\n'
    * treti obsahuje stejnou chybu jako o bod vyse, ovsem opakovana veta je zopakovana 250 krat
    * ctvrty obsahuje stejnou chybu jako prvni zmineny radek

In [25]:
df[df['Medium'].str.len() > 1000].Medium

38106     Carnelian and white agate\r\n\r\n\r\n\r\n\r\n\...
207191    folio 1 -     Pen and brown ink, brush and bro...
211584    1)     Pen and brown ink, brush and gray wash....
308138    Silver; micro-repoussage (sheet metal miniatur...
Name: Medium, dtype: object

## 6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature

* V bode 2) jsem chybejici hodnoty ve sloupci 'Object Name' zpracoval a vsechny zaradil pod typ 'Other'
* V bode 4) jsem upravil navic nekolik sloupcu, z nichz sloupec 'Classification' jsem categorizoval a rozhodl jsem se chybejici hodnoty take zaradit do kategorie 'Other'
* Dalsi chybejici data jsem odhalil ve sloupcich 'Dynasty' a 'State'.
    * Hodnotu ve sloupci 'State'jsem se pokousel nahradit nasledovne: ( bohuzel po implementovani jsem zjistil, ze takove radky neexistuji :( )
        * Pokud je v datasetu radek, ktery obsahuje stejnou hodnotu ve sloupci 'City' a ma 'State' vypleny, tak tuto hodnotu zkopiruji do radku, kteremu hodnota 'State' chybi
    * Hodnota ve sloupci 'Dynasty' se sice muze zdat jako 'schvalne nevyplena', ale kdyz se vice prozkoumaji vyplnene hodnoty, tak se casto vyskytuje i pouhy odhad, do ktere dynastie dilo patri (na konci hodnoty dynastie je '(?)')

In [26]:
# # print(df['State'].unique())
# # ve sloupci se vyskutij dve hodnoty (| a ||), ktere jsou zrejme chybne. Nejprve je tedy smazu
# df.loc[df['State'] == '|', 'State'] = np.nan
# df.loc[df['State'] == '||', 'State'] = np.nan
# cityToStateDic = {}

In [27]:
# cityToStateDic = {}
# for index, row in df.iterrows():
#     if str(row.City) not in cityToStateDic.keys() and str(row.State) != 'nan' and len(str(row.State)) > 0:
#         cityToStateDic[str(row.City)] = row.State

In [28]:
# dfStateToFill = df[df['State'].isnull()]
# for index, row in dfStateToFill.iterrows():
#     if str(row.City) in cityToStateDic.keys():
#         row.State = cityToStateDic[str(row.City)]
#     else:
#         row.State = np.nan

In [29]:
# print('Len of rows with missing \'State\' before:', len(df[df['State'].isnull()]))
# df[df['State'].isnull()]['State'] = dfStateToFill['State']
# print('Len of rows with missing \'State\' after:', len(df[df['State'].isnull()]))

* Ve sloupci 'AccessionYear' chybi pomerne malo hodnot. Rozhodl jsem se je doplnit pomoci mean hodnoty. Vsechny vyplnene hodnoty jsou totiz z podobneho obdobi (rozmezi 100 let), proto si myslim, ze mean hodnota je dobra volba.

In [30]:
# nejdriv jsem musel upravit hodnoty do celo ciselne podoby
def convertoToYearInt(val):
    val_str = str(val)
    if val_str.isnumeric() or type(val) is float and not np.isnan(val):
        return int(val)
    elif val_str[:4].isnumeric():
        return int(val[:4])
    else:
        return val

In [31]:
df['AccessionYear'] = df['AccessionYear'].apply(convertoToYearInt)

In [32]:
# df['AccessionYear'].unique()
AccessionYear_mean = df[df['AccessionYear'].notnull()]['AccessionYear'].mean()
print('Replacing with mean value:', AccessionYear_mean)
df['AccessionYear'].fillna(AccessionYear_mean, inplace=True)

Replacing with mean value: 1955.3089548908135
